In [7]:
from PIL import Image
import numpy as np
import os
import pickle
import random
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler
import umap

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from torchvision import transforms
from tqdm import tqdm

# Use IoU instead
# import re
from scipy.spatial.distance import cdist
from skimage.transform import resize
from time import time


# Read the original masks

In [8]:
synthia_sample_number = 500 # or 500

In [9]:
# the website folder, we make relative folder based on this as the source folder
website_folder = "D:\\DomainVis\\Graduation-project-domain-shift-image-2-image"
dir_path = os.path.join(website_folder,"dataset\\domain_adaptation")
cityscapes_path = os.path.join(dir_path,"Cityscapes")
synthia_path = os.path.join(dir_path,"Synthia")

# note that each folder for synthia contains subfolders of citynames.
cityscapes_image_folder = os.path.join(cityscapes_path,"leftImg8bit\\val")
cityscapes_label_folder = os.path.join(cityscapes_path,"gtFine\\val") # also need to find the right labels
cityscapes_predictions_folder = os.path.join(cityscapes_path,"outputs\\preds")

if synthia_sample_number:
    synthia_image_folder = os.path.join(synthia_path,"RGB_subset_")+str(synthia_sample_number) # all the images for now
else:
    synthia_image_folder = os.path.join(synthia_path,"RGB")

# color mask
synthia_label_image_folder = os.path.join(synthia_path,"GT\\COLOR") # also need to find the right labels
# label class folder 
synthia_label_class_folder = os.path.join(synthia_path,"GT\\LABELS")
synthia_predictions_folder = os.path.join(synthia_path,"predictions_")+str(synthia_sample_number)+"\\predictions"

In [11]:
print(cityscapes_image_folder)

D:\DomainVis\Graduation-project-domain-shift-image-2-image\dataset\domain_adaptation\Cityscapes\leftImg8bit\val


In [12]:
# root of the images
root_image_folder = cityscapes_image_folder
root_label_folder = cityscapes_label_folder
root_predictions_folder = cityscapes_predictions_folder
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']

cityscapes_full_paths = []
# cityscapes_names = []
# cityscapes_rel_paths = []
cityscapes_initial_info = []
cityscapes_images = []
cityscapes_labels = []
cityscapes_predictions = []

# Walk through root folder and subfolders
for subdir, dirs, files in os.walk(root_image_folder):
    for file in files:
        # Check if the file has one of the desired extensions
        if any(file.lower().endswith(ext) for ext in image_extensions):
            # path of image
            full_image_path = os.path.join(subdir, file)
            rel_image_path = os.path.relpath(full_image_path, website_folder)
            # read in the images
            image = Image.open(full_image_path).convert("RGB")
            cityscapes_images.append(np.array(image))
            
            # path of corresponding label (color form)
            subfolder = file.split("_")[0] # the first word in the file corresponds to subfolder name
            parsed_name_parts = file.split("_")[:3]
            label_name = "_".join(parsed_name_parts)+"_gtFine_color.png"
#             label_name = label_name + 
            full_label_path = os.path.join(root_label_folder,subfolder, label_name)
            rel_label_path = os.path.relpath(full_label_path, website_folder)
#             label_class_path = "D:\\zsh\\graduation\\DAFormer\\data\\Cityscapes\\gtFine\\val\\frankfurt\\frankfurt_000000_000294_gtFine_labelIds.png"
            label_class_name = "_".join(parsed_name_parts)+"_gtFine_labelIds.png"
            label_class_path = os.path.join(root_label_folder,subfolder, label_class_name)
            label = Image.open(label_class_path).convert("L")
            cityscapes_labels.append(np.array(label))
            
            # path of predictions
            pred_name = file # same name as the input
            full_pred_path = os.path.join(root_predictions_folder,subfolder, pred_name)
            rel_pred_path = os.path.relpath(full_pred_path, website_folder)
            prediction = Image.open(full_pred_path).convert("RGB")# load predictions
            cityscapes_predictions.append(np.array(prediction))
            
            # add the paths
            cityscapes_full_paths.append((full_image_path,full_label_path))
            cityscapes_initial_info.append((file,rel_image_path,rel_label_path,rel_pred_path,"Cityscapes"))

# Print the results
print(cityscapes_full_paths[0])
print(cityscapes_initial_info[0]) # len 500

('D:\\DomainVis\\Graduation-project-domain-shift-image-2-image\\dataset\\domain_adaptation\\Cityscapes\\leftImg8bit\\val\\frankfurt\\frankfurt_000000_000294_leftImg8bit.png', 'D:\\DomainVis\\Graduation-project-domain-shift-image-2-image\\dataset\\domain_adaptation\\Cityscapes\\gtFine\\val\\frankfurt\\frankfurt_000000_000294_gtFine_color.png')
('frankfurt_000000_000294_leftImg8bit.png', 'dataset\\domain_adaptation\\Cityscapes\\leftImg8bit\\val\\frankfurt\\frankfurt_000000_000294_leftImg8bit.png', 'dataset\\domain_adaptation\\Cityscapes\\gtFine\\val\\frankfurt\\frankfurt_000000_000294_gtFine_color.png', 'dataset\\domain_adaptation\\Cityscapes\\outputs\\preds\\frankfurt\\frankfurt_000000_000294_leftImg8bit.png', 'Cityscapes')


In [22]:
imageio.plugins.freeimage.download() # very important!!!

Imageio: 'freeimage-3.15.1-win64.dll' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/freeimage/FreeImage-3.15.1-win64.dll (3.4 MB)
Downloading: 8192/3527168 bytes (0.2%3527168/3527168 bytes (100.0%)
  Done
File saved as C:\Users\20211016\AppData\Local\imageio\freeimage\freeimage-3.15.1-win64.dll.


In [25]:
# import imageio.v2 as imageio
import imageio

root_image_folder = synthia_image_folder
root_label_folder = synthia_label_image_folder
root_class_folder = synthia_label_class_folder
root_predictions_folder = synthia_predictions_folder

synthia_names = os.listdir(synthia_image_folder)
synthia_full_paths = []
synthia_images =[]
synthia_labels = []
synthia_predictions = []
synthia_initial_info = []

for file in synthia_names:
    # path of image
    full_image_path = os.path.join(root_image_folder,file)
    rel_image_path = os.path.relpath(full_image_path, website_folder)
    image = Image.open(full_image_path).convert("RGB")
    synthia_images.append(np.array(image))
    
    # path of corresponding label
    full_label_path = os.path.join(root_label_folder,file)
    rel_label_path = os.path.relpath(full_label_path, website_folder)

    # parsed_name = file.split(".")[0]
    # label_class_name = parsed_name+"_labelTrainIds.png"
    # label_class_path = os.path.join(root_class_folder,label_class_name)
    label_class_path = os.path.join(root_class_folder,file)
    # label_image = cv2.imread(label_class_path, cv2.IMREAD_UNCHANGED)
    label_image = imageio.imread(label_class_path,format="PNG-FI")
    label = np.asarray(label_image)[:,:,0]
    
    synthia_labels.append(np.array(label))
    
    # path of corresponding predictions
    full_pred_path = os.path.join(root_predictions_folder,file)
    rel_pred_path = os.path.relpath(full_pred_path, website_folder)
    prediction = Image.open(full_pred_path).convert("RGB")# load predictions
    synthia_predictions.append(np.array(prediction))
    
    synthia_full_paths.append((full_image_path,full_label_path))
    synthia_initial_info.append((file,rel_image_path,rel_label_path,rel_pred_path,"Synthia"))
# Print the results
print(synthia_full_paths[0])
print(synthia_initial_info[0])

('D:\\DomainVis\\Graduation-project-domain-shift-image-2-image\\dataset\\domain_adaptation\\Synthia\\RGB_subset_500\\0000022.png', 'D:\\DomainVis\\Graduation-project-domain-shift-image-2-image\\dataset\\domain_adaptation\\Synthia\\GT\\COLOR\\0000022.png')
('0000022.png', 'dataset\\domain_adaptation\\Synthia\\RGB_subset_500\\0000022.png', 'dataset\\domain_adaptation\\Synthia\\GT\\COLOR\\0000022.png', 'dataset\\domain_adaptation\\Synthia\\predictions_500\\predictions\\0000022.png', 'Synthia')


In [20]:
!pip install --upgrade imageio

# Labels to classes

In [26]:
# Cityscpaes
cityscapes_labels_arr = np.array(cityscapes_labels)
print(cityscapes_labels_arr.shape)

(500, 1024, 2048)


In [27]:
cityscapes_labels_arr[0]

array([[3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       ...,
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3]], dtype=uint8)

In [28]:
new_class_names = ["unlabelled","road","sidewalk","vegetation","sky","car"]
# mapping the classes to 6 classes
# when using color to class, 0-3 is ignored
category_map = {
    0: 0,
    1: 0,
    2: 0,
    3: 0,
    4: 0,
    5: 0,
    6: 0,
    7: 1,
    8: 2,
    9: 0,
    10: 0,
    11: 0,
    12: 0,
    13: 0,
    14: 0,
    15: 0,
    16: 0,
    17: 0,
    18: 0,
    19: 0,
    20: 0,
    21: 3,
    22: 0,
    23: 4,
    24: 0,
    25: 0,
    26: 5,
    27: 0,
    28: 0,
    29: 0,
    30: 0,
    31: 0,
    32: 0,
    33: 0
}

In [29]:
# transform a colorful mask image to mask with classes
def map_to_6_classes(label,categorical_map):
    mask = np.zeros(shape=(label.shape[0], label.shape[1]), dtype = np.int32)
    for row in range(label.shape[0]):
        for col in range(label.shape[1]):
            idx = label[row, col]
            # d = cdist(np.array([a]),np.array(cityscapes_colors))
            # idx = np.argmin(d)
            new_idx = categorical_map[idx]
            mask[row, col] = new_idx
    mask = np.reshape(mask, (mask.shape[0], mask.shape[1]))
    return mask

In [30]:
new_class_names = ["unlabelled","road","sidewalk","vegetation","sky","car"]

category_map_synthia = {
    0: 0,
    1: 4,
    2: 0,
    3: 1,
    4: 2,
    5: 0,
    6: 3,
    7: 0,
    8: 5,
    9: 0,
    10: 0,
    11: 0,
    12: 0,
    13: 0,
    14: 0,
    15: 0,
    16: 0,
    17: 0,
    18: 0,
    19: 0,
    20: 0,
    21: 3,
    22: 0,
    255:0,
}

In [31]:
start_time = time()
cityscapes_labels_6_classes = []
for i,label in enumerate(cityscapes_labels_arr):
    if i%50==0:
        print(i)
    new_label = map_to_6_classes(label,category_map)
    cityscapes_labels_6_classes.append(new_label)
end_time = time()
print(end_time-start_time) # 885 s = 15 min

0
50
100
150
200
250
300
350
400
450
817.2240478992462


In [32]:
synthia_labels_arr = np.array(synthia_labels)
print(synthia_labels_arr.shape)

start_time = time()
synthia_labels_6_classes = []
for i,label in enumerate(synthia_labels_arr):
    if i%50==0:
        print(i)
    new_label = map_to_6_classes(label,category_map_synthia)
    synthia_labels_6_classes.append(new_label)
end_time = time()
print(end_time-start_time) # 

print(synthia_labels_6_classes[0])

(500, 760, 1280)
0
50
100
150
200
250
300
350
400
450
386.0364303588867
[[0 0 0 ... 4 4 4]
 [0 0 0 ... 4 4 4]
 [0 0 0 ... 4 4 4]
 ...
 [1 1 1 ... 2 2 2]
 [1 1 1 ... 2 2 2]
 [1 1 1 ... 2 2 2]]


# Test out color encoding with sample images

In [33]:
# randomly choose 10 images for each datasec
cityscape_labels_sample = cityscapes_labels_6_classes[:5]

In [34]:
np.unique(cityscape_labels_sample[0])

array([0, 1, 2, 3, 4, 5])

In [38]:
cityscape_labels_sample[0].shape

(1024, 2048)

In [35]:
print(cityscape_labels_sample[0])

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [47]:
# class to color: for viewing labels
# selected_light_colors = {
#     "others": [222, 203, 228],
#     "road": [254, 217, 166],
#     "sidewalk": [255, 255, 204],
#     "vegetation": [229, 216, 189],
#     "sky": [253, 218, 236],
#     "car": [251, 180, 174]
# }
selected_light_colors = {
    "others": [210, 157, 227],
    "road": [255, 196, 115],
    "sidewalk": [255, 255, 153],
    "vegetation": [230, 202, 145],
    "sky": [252, 167, 211],
    "car": [240, 130, 120]
}
class_names = ["others","road","sidewalk","vegetation","sky","car"]

def class_to_color(labels,dataset):
    if dataset =="Synthia":
        label_colors = np.zeros((760,1280,3))
    else:
        label_colors = np.zeros((1024,2048,3))
    
    for i,row in enumerate(labels):
        for j,pixel in enumerate(row):
            # print(pixel, type(pixel))
            class_name = class_names[pixel]
            label_colors[i,j] = selected_light_colors[class_name]
    
    return label_colors.astype(np.uint8)  # make each pixel value an integer to visualize it better

In [37]:
from IPython.display import display

In [38]:
import matplotlib.pyplot as plt

In [39]:
cityscapes_initial_info[i][0]

'munster_000173_000019_leftImg8bit.png'

In [45]:
website_folder = "D:\\DomainVis\\Graduation-project-domain-shift-image-2-image"

In [56]:
new_labels_path ="D:\\DomainVis\\Graduation-project-domain-shift-image-2-image\\dataset\\domain_adaptation\\Cityscapes\\New_labels_v2"
new_label_paths_cityscapes = []

for i, label in enumerate(cityscapes_labels_6_classes):
#     if i==9:
#         break
    name = cityscapes_initial_info[i][0]
    new_label_arr = class_to_color(label,"Cityscapes")
    new_label_img = Image.fromarray(new_label_arr, 'RGB')
    # Display the image
    new_saved_path = os.path.join(new_labels_path,name)
    new_label_img.save(new_saved_path)

    new_rel_path = os.path.relpath(new_saved_path, website_folder)
    new_label_paths_cityscapes.append(new_rel_path)

In [57]:
new_labels_path_synthia ="D:\\DomainVis\\Graduation-project-domain-shift-image-2-image\\dataset\\domain_adaptation\\Synthia\\new_labels_v2"
new_label_paths_synthia = []

for i, label in enumerate(synthia_labels_6_classes):
#     if i==9:
#         break
    name = synthia_initial_info[i][0]
    new_label_arr = class_to_color(label,dataset="Synthia")
    new_label_img = Image.fromarray(new_label_arr, 'RGB')
    # Display the image
    new_saved_path = os.path.join(new_labels_path_synthia,name)
    new_label_img.save(new_saved_path)

    new_rel_path = os.path.relpath(new_saved_path, website_folder)
    new_label_paths_synthia.append(new_rel_path)

In [58]:
print(new_label_arr[0])

[[210 157 227]
 [210 157 227]
 [210 157 227]
 ...
 [210 157 227]
 [210 157 227]
 [210 157 227]]


In [59]:
new_label_arr.dtype

dtype('uint8')

In [60]:
np.array(new_label_paths_cityscapes).shape

(500,)

In [61]:
new_label_paths_combined = np.hstack((np.array(new_label_paths_cityscapes),np.array(new_label_paths_synthia)))

In [62]:
new_label_paths_combined.shape #(1000,0)

(1000,)

In [63]:
new_label_paths_combined[0]

'dataset\\domain_adaptation\\Cityscapes\\New_labels_v2\\frankfurt_000000_000294_leftImg8bit.png'

In [66]:
# read in the old df, and remove the label paths column
original_path = "D:\\DomainVis\\Graduation-project-domain-shift-image-2-image\\discreted_data_v16.csv"
original_df= pd.read_csv(original_path,index_col = "id")

In [67]:
new_df = original_df.copy()
new_df.drop(columns = ["label_path"])

,name,dataset,image_path,prediction_path,similar_image_paths,similar_IoU_score,other_ratio,road_ratio,sidewalk_ratio,vegetation_ratio,...,similar_image_paths_road,similar_IoU_score_road,similar_image_paths_sidewalk,similar_IoU_score_sidewalk,similar_image_paths_vegetation,similar_IoU_score_vegetation,similar_image_paths_sky,similar_IoU_score_sky,similar_image_paths_car,similar_IoU_score_car
id,,,,,,,,,,,,,,,,,,,,,
0,frankfurt_000000_000294_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.660623,0.528985,0.297838,0.080459,0.020084,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.669665,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.445261,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.526673,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.516886,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.220297
1,frankfurt_000000_000576_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.581997,0.384708,0.345542,0.019122,0.085221,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.625078,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.192217,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.338053,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.546513,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.328769
2,frankfurt_000000_001016_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.663334,0.505177,0.336215,0.023324,0.011662,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.603962,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.205325,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.126580,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.000000,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.309487
3,frankfurt_000000_001236_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.484486,0.179743,0.381479,0.009730,0.256977,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.685399,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.090810,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.425622,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.459195,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.343273
4,frankfurt_000000_001751_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.659385,0.405927,0.379869,0.024012,0.052703,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.682732,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.358655,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.569467,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.642079,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.213340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0009354.png,Synthia,dataset\domain_adaptation\Synthia\RGB_subset_5...,dataset\domain_adaptation\Synthia\predictions_...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.472945,0.340820,0.336541,0.197154,0.094768,...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.414943,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.326770,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.252685,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.000346,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.053920
996,0009359.png,Synthia,dataset\domain_adaptation\Synthia\RGB_subset_5...,dataset\domain_adaptation\Synthia\predictions_...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.380413,0.284762,0.331777,0.245871,0.107303,...,dataset\domain_ada

In [68]:
# Save the new label paths to new version of df
new_df["label_path"]=new_label_paths_combined

In [69]:
new_df

,name,dataset,image_path,label_path,prediction_path,similar_image_paths,similar_IoU_score,other_ratio,road_ratio,sidewalk_ratio,...,similar_image_paths_road,similar_IoU_score_road,similar_image_paths_sidewalk,similar_IoU_score_sidewalk,similar_image_paths_vegetation,similar_IoU_score_vegetation,similar_image_paths_sky,similar_IoU_score_sky,similar_image_paths_car,similar_IoU_score_car
id,,,,,,,,,,,,,,,,,,,,,
0,frankfurt_000000_000294_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.660623,0.528985,0.297838,0.080459,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.669665,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.445261,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.526673,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.516886,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.220297
1,frankfurt_000000_000576_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.581997,0.384708,0.345542,0.019122,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.625078,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.192217,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.338053,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.546513,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.328769
2,frankfurt_000000_001016_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.663334,0.505177,0.336215,0.023324,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.603962,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.205325,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.126580,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.000000,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.309487
3,frankfurt_000000_001236_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.484486,0.179743,0.381479,0.009730,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.685399,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.090810,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.425622,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.459195,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.343273
4,frankfurt_000000_001751_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.659385,0.405927,0.379869,0.024012,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.682732,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.358655,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.569467,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.642079,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.213340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0009354.png,Synthia,dataset\domain_adaptation\Synthia\RGB_subset_5...,dataset\domain_adaptation\Synthia\new_labels_v...,dataset\domain_adaptation\Synthia\predictions_...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.472945,0.340820,0.336541,0.197154,...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.414943,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.326770,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.252685,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.000346,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.053920
9

In [70]:
new_df.to_csv("discreted_data_v17.csv",index = "id")

# Prediction

In [71]:
cityscapes_colors = [[  0,   0,   0], # this include five classes, but only one could be matched
         [111, 74,  0], # dynamic
         [81,0,81], # ground
        [128, 64, 128], # road
        [244, 35, 232], # sidewalk
         [250,170,160], # parking
         [230,150,140], # railtrack
        [70, 70, 70], # building
        [102, 102, 156],# wall
        [190, 153, 153],# fence
         [180,165,180], # gard rail
         [150,100,100], # bridge
         [150,120,90], # tunnel
        [153, 153, 153], # pole
         [153, 153, 153],# polegroup
        [250, 170, 30], # traffic light
        [220, 220, 0],  # traffic sign
        [107, 142, 35], # vegetation
        [152, 251, 152], # terrain
        [70, 130, 180], # sky
        [220, 20, 60], # person
        [255, 0, 0], # rider
        [0, 0, 142], # car
        [0, 0, 70], # truck
        [0, 60, 100], # bus
         [0,0,90], # caravan
         [ 0, 0,110], # trailor
        [0, 80, 100], # train
        [0, 0, 230], # motorcycle
        [119, 11, 32], # bicycle
    ]

In [72]:
def find_index_2d(array, subarray):
    for index, row in enumerate(array):
        if np.array_equal(row, subarray):
            return index
    return -1  # return -1 if the subarray is not found

def color_to_class(label,categorical_map,colors,dataset_name):
    # create new empty mask
    mask = np.zeros(shape=(label.shape[0], label.shape[1]), dtype = np.int32)
    # iterate through two dimensions
    for row in range(label.shape[0]):
        for col in range(label.shape[1]):
            a = label[row, col,:]
            # distance between this pixel and the original pixel
            # d = cdist(np.array([a]),np.array(synthia_colors))
            # idx = np.argmin(d)
            idx = find_index_2d(colors,a)
            if dataset_name == "Cityscapes":
                new_idx = categorical_map[idx+4]
            elif dataset_name == "Synthia":
                new_idx = categorical_map[idx]
            mask[row, col] = new_idx
    mask = np.reshape(mask, (mask.shape[0], mask.shape[1]))
    return mask

In [77]:
new_predictions_path ="D:\\DomainVis\\Graduation-project-domain-shift-image-2-image\\dataset\\domain_adaptation\\Cityscapes\\outputs\\preds_new_color_v2"
new_pred_paths_cityscapes = []

for i, prediction in enumerate(cityscapes_predictions):
#     start_time = time()
    name = cityscapes_initial_info[i][0]
    # read in the color map 
    prediction = cityscapes_predictions[i]
    prediction_mask = color_to_class(prediction,category_map,cityscapes_colors,"Cityscapes")

    # transform the class mask to new color
    new_label_arr = class_to_color(prediction_mask,"Cityscapes")
    new_label_img = Image.fromarray(new_label_arr, 'RGB')

    # save the image
    new_saved_path = os.path.join(new_predictions_path,name)
    new_label_img.save(new_saved_path)

    new_rel_path = os.path.relpath(new_saved_path, website_folder)
    new_pred_paths_cityscapes.append(new_rel_path)
#     end_time = time()
#     print(end_time-start_time)

In [172]:
new_pred_paths_cityscapes[:3]

['dataset\\domain_adaptation\\Cityscapes\\outputs\\preds_new_color\\frankfurt_000000_000294_leftImg8bit.png',
 'dataset\\domain_adaptation\\Cityscapes\\outputs\\preds_new_color\\frankfurt_000000_000576_leftImg8bit.png',
 'dataset\\domain_adaptation\\Cityscapes\\outputs\\preds_new_color\\frankfurt_000000_001016_leftImg8bit.png']

In [78]:
new_predictions_path_synthia ="D:\\DomainVis\\Graduation-project-domain-shift-image-2-image\\dataset\\domain_adaptation\\Synthia\\Predictions_500\\preds_new_color_v2"
new_pred_paths_synthia = []

for i, label in enumerate(synthia_predictions):
    name = synthia_initial_info[i][0]
    prediction = synthia_predictions[i]
    prediction_mask = color_to_class(prediction,category_map,cityscapes_colors,"Cityscapes")
    print(np.unique(prediction_mask))

    # transform the class mask to new color
    new_label_arr = class_to_color(prediction_mask,dataset="Synthia")
    new_label_img = Image.fromarray(new_label_arr, 'RGB')
    
    # save the image
    new_saved_path = os.path.join(new_predictions_path_synthia,name)
    new_label_img.save(new_saved_path)

    new_rel_path = os.path.relpath(new_saved_path, website_folder)
    new_pred_paths_synthia.append(new_rel_path)

[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 5]
[0 1 2 3 4]
[0 1 2 3 4 5]
[0 1 2 3 4]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]


In [174]:
new_pred_paths_synthia[:3]

['dataset\\domain_adaptation\\Synthia\\predictions_500\\preds_new_color\\0000022.png',
 'dataset\\domain_adaptation\\Synthia\\predictions_500\\preds_new_color\\0000026.png',
 'dataset\\domain_adaptation\\Synthia\\predictions_500\\preds_new_color\\0000037.png']

In [79]:
new_pred_paths_cityscapes.extend(new_pred_paths_synthia) # now new pred_paths include both cityscapes and synthia prediction paths

In [80]:
new_pred_paths_cityscapes[495:505] # Important: Note that this is the combined version!!!

['dataset\\domain_adaptation\\Cityscapes\\outputs\\preds_new_color_v2\\munster_000169_000019_leftImg8bit.png',
 'dataset\\domain_adaptation\\Cityscapes\\outputs\\preds_new_color_v2\\munster_000170_000019_leftImg8bit.png',
 'dataset\\domain_adaptation\\Cityscapes\\outputs\\preds_new_color_v2\\munster_000171_000019_leftImg8bit.png',
 'dataset\\domain_adaptation\\Cityscapes\\outputs\\preds_new_color_v2\\munster_000172_000019_leftImg8bit.png',
 'dataset\\domain_adaptation\\Cityscapes\\outputs\\preds_new_color_v2\\munster_000173_000019_leftImg8bit.png',
 'dataset\\domain_adaptation\\Synthia\\Predictions_500\\preds_new_color_v2\\0000022.png',
 'dataset\\domain_adaptation\\Synthia\\Predictions_500\\preds_new_color_v2\\0000026.png',
 'dataset\\domain_adaptation\\Synthia\\Predictions_500\\preds_new_color_v2\\0000037.png',
 'dataset\\domain_adaptation\\Synthia\\Predictions_500\\preds_new_color_v2\\0000042.png',
 'dataset\\domain_adaptation\\Synthia\\Predictions_500\\preds_new_color_v2\\0000052.p

In [88]:
df_old = pd.read_csv("discreted_data_v17.csv",index_col="id")
df_old.head()

,name,dataset,image_path,label_path,prediction_path,similar_image_paths,similar_IoU_score,other_ratio,road_ratio,sidewalk_ratio,...,similar_image_paths_road,similar_IoU_score_road,similar_image_paths_sidewalk,similar_IoU_score_sidewalk,similar_image_paths_vegetation,similar_IoU_score_vegetation,similar_image_paths_sky,similar_IoU_score_sky,similar_image_paths_car,similar_IoU_score_car
id,,,,,,,,,,,,,,,,,,,,,
0,frankfurt_000000_000294_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.660623,0.528985,0.297838,0.080459,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.669665,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.445261,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.526673,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.516886,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.220297
1,frankfurt_000000_000576_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.581997,0.384708,0.345542,0.019122,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.625078,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.192217,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.338053,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.546513,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.328769
2,frankfurt_000000_001016_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.663334,0.505177,0.336215,0.023324,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.603962,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.205325,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.126580,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.000000,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.309487
3,frankfurt_000000_001236_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.484486,0.179743,0.381479,0.009730,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.685399,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.090810,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.425622,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.459195,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.343273
4,frankfurt_000000_001751_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.659385,0.405927,0.379869,0.024012,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.682732,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.358655,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.569467,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.642079,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.213340


In [89]:
df_old["label_path"].iloc[0]

'dataset\\domain_adaptation\\Cityscapes\\New_labels_v2\\frankfurt_000000_000294_leftImg8bit.png'

In [93]:
new_df = df_old.copy()
new_df.drop(columns = ["prediction_path"])

,name,dataset,image_path,label_path,similar_image_paths,similar_IoU_score,other_ratio,road_ratio,sidewalk_ratio,vegetation_ratio,...,similar_image_paths_road,similar_IoU_score_road,similar_image_paths_sidewalk,similar_IoU_score_sidewalk,similar_image_paths_vegetation,similar_IoU_score_vegetation,similar_image_paths_sky,similar_IoU_score_sky,similar_image_paths_car,similar_IoU_score_car
id,,,,,,,,,,,,,,,,,,,,,
0,frankfurt_000000_000294_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.660623,0.528985,0.297838,0.080459,0.020084,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.669665,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.445261,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.526673,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.516886,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.220297
1,frankfurt_000000_000576_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.581997,0.384708,0.345542,0.019122,0.085221,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.625078,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.192217,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.338053,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.546513,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.328769
2,frankfurt_000000_001016_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.663334,0.505177,0.336215,0.023324,0.011662,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.603962,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.205325,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.126580,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.000000,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.309487
3,frankfurt_000000_001236_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.484486,0.179743,0.381479,0.009730,0.256977,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.685399,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.090810,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.425622,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.459195,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.343273
4,frankfurt_000000_001751_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.659385,0.405927,0.379869,0.024012,0.052703,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.682732,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.358655,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.569467,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.642079,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.213340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0009354.png,Synthia,dataset\domain_adaptation\Synthia\RGB_subset_5...,dataset\domain_adaptation\Synthia\new_labels_v...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.472945,0.340820,0.336541,0.197154,0.094768,...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.414943,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.326770,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.252685,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.000346,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.053920
996,0009359.png,Synthia,dataset\domain_adaptation\Synthia\RGB_subset_5...,dataset\domain_adaptation\Synthia\new_labels_v...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.380413,0.284762,0.331777,0.245871,0.107303,...,dataset\domain_adaptati

In [94]:
new_df["prediction_path"]=new_pred_paths_cityscapes

In [97]:
new_df

,name,dataset,image_path,label_path,prediction_path,similar_image_paths,similar_IoU_score,other_ratio,road_ratio,sidewalk_ratio,...,similar_image_paths_road,similar_IoU_score_road,similar_image_paths_sidewalk,similar_IoU_score_sidewalk,similar_image_paths_vegetation,similar_IoU_score_vegetation,similar_image_paths_sky,similar_IoU_score_sky,similar_image_paths_car,similar_IoU_score_car
id,,,,,,,,,,,,,,,,,,,,,
0,frankfurt_000000_000294_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.660623,0.528985,0.297838,0.080459,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.669665,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.445261,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.526673,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.516886,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.220297
1,frankfurt_000000_000576_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.581997,0.384708,0.345542,0.019122,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.625078,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.192217,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.338053,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.546513,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.328769
2,frankfurt_000000_001016_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.663334,0.505177,0.336215,0.023324,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.603962,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.205325,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.126580,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.000000,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.309487
3,frankfurt_000000_001236_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.484486,0.179743,0.381479,0.009730,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.685399,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.090810,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.425622,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.459195,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.343273
4,frankfurt_000000_001751_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.659385,0.405927,0.379869,0.024012,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.682732,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.358655,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.569467,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.642079,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.213340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0009354.png,Synthia,dataset\domain_adaptation\Synthia\RGB_subset_5...,dataset\domain_adaptation\Synthia\new_labels_v...,dataset\domain_adaptation\Synthia\Predictions_...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.472945,0.340820,0.336541,0.197154,...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.414943,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.326770,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.252685,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.000346,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.053920
9

In [99]:
new_df["prediction_path"].iloc[0]

'dataset\\domain_adaptation\\Cityscapes\\outputs\\preds_new_color_v2\\frankfurt_000000_000294_leftImg8bit.png'

In [98]:
new_df.to_csv("discreted_data_v18.csv",index = "id")

# combine the two paths and add to current df

In [158]:
new_label_paths_synthia[-1]

'dataset\\domain_adaptation\\Synthia\\new labels\\0000022.png'

In [137]:
print(end_time-start_time)

39.345325231552124


# Read the original data in

In [99]:
df_old = pd.read_csv("discreted_data_v16.csv",index_col="id")
df_old.head()

,name,dataset,image_path,label_path,prediction_path,similar_image_paths,similar_IoU_score,other_ratio,road_ratio,sidewalk_ratio,...,similar_image_paths_road,similar_IoU_score_road,similar_image_paths_sidewalk,similar_IoU_score_sidewalk,similar_image_paths_vegetation,similar_IoU_score_vegetation,similar_image_paths_sky,similar_IoU_score_sky,similar_image_paths_car,similar_IoU_score_car
id,,,,,,,,,,,,,,,,,,,,,
0,frankfurt_000000_000294_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\gtFine\va...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.660623,0.528985,0.297838,0.080459,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.669665,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.445261,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.526673,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.516886,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.220297
1,frankfurt_000000_000576_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\gtFine\va...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.581997,0.384708,0.345542,0.019122,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.625078,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.192217,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.338053,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.546513,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.328769
2,frankfurt_000000_001016_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\gtFine\va...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.663334,0.505177,0.336215,0.023324,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.603962,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.205325,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.126580,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.000000,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.309487
3,frankfurt_000000_001236_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\gtFine\va...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.484486,0.179743,0.381479,0.009730,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.685399,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.090810,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.425622,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.459195,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.343273
4,frankfurt_000000_001751_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\gtFine\va...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.659385,0.405927,0.379869,0.024012,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.682732,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.358655,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.569467,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.642079,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.213340


In [114]:
df_old.drop(columns = ["label_path"])

,name,dataset,image_path,prediction_path,similar_image_paths,similar_IoU_score,other_ratio,road_ratio,sidewalk_ratio,vegetation_ratio,...,similar_image_paths_road,similar_IoU_score_road,similar_image_paths_sidewalk,similar_IoU_score_sidewalk,similar_image_paths_vegetation,similar_IoU_score_vegetation,similar_image_paths_sky,similar_IoU_score_sky,similar_image_paths_car,similar_IoU_score_car
id,,,,,,,,,,,,,,,,,,,,,
0,frankfurt_000000_000294_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.660623,0.528985,0.297838,0.080459,0.020084,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.669665,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.445261,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.526673,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.516886,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.220297
1,frankfurt_000000_000576_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.581997,0.384708,0.345542,0.019122,0.085221,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.625078,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.192217,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.338053,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.546513,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.328769
2,frankfurt_000000_001016_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.663334,0.505177,0.336215,0.023324,0.011662,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.603962,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.205325,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.126580,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.000000,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.309487
3,frankfurt_000000_001236_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.484486,0.179743,0.381479,0.009730,0.256977,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.685399,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.090810,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.425622,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.459195,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.343273
4,frankfurt_000000_001751_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.659385,0.405927,0.379869,0.024012,0.052703,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.682732,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.358655,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.569467,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.642079,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.213340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0009354.png,Synthia,dataset\domain_adaptation\Synthia\RGB_subset_5...,dataset\domain_adaptation\Synthia\predictions_...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.472945,0.340820,0.336541,0.197154,0.094768,...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.414943,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.326770,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.252685,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.000346,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.053920
996,0009359.png,Synthia,dataset\domain_adaptation\Synthia\RGB_subset_5...,dataset\domain_adaptation\Synthia\predictions_...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.380413,0.284762,0.331777,0.245871,0.107303,...,dataset\domain_ada

In [115]:
df_old["label_path"] = new_label_paths_combined.tolist()

In [116]:
df_old

,name,dataset,image_path,label_path,prediction_path,similar_image_paths,similar_IoU_score,other_ratio,road_ratio,sidewalk_ratio,...,similar_image_paths_road,similar_IoU_score_road,similar_image_paths_sidewalk,similar_IoU_score_sidewalk,similar_image_paths_vegetation,similar_IoU_score_vegetation,similar_image_paths_sky,similar_IoU_score_sky,similar_image_paths_car,similar_IoU_score_car
id,,,,,,,,,,,,,,,,,,,,,
0,frankfurt_000000_000294_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.660623,0.528985,0.297838,0.080459,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.669665,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.445261,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.526673,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.516886,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.220297
1,frankfurt_000000_000576_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.581997,0.384708,0.345542,0.019122,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.625078,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.192217,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.338053,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.546513,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.328769
2,frankfurt_000000_001016_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.663334,0.505177,0.336215,0.023324,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.603962,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.205325,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.126580,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.000000,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.309487
3,frankfurt_000000_001236_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.484486,0.179743,0.381479,0.009730,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.685399,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.090810,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.425622,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.459195,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.343273
4,frankfurt_000000_001751_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.659385,0.405927,0.379869,0.024012,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.682732,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.358655,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.569467,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.642079,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.213340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0009354.png,Synthia,dataset\domain_adaptation\Synthia\RGB_subset_5...,dataset\domain_adaptation\Synthia\new labels\0...,dataset\domain_adaptation\Synthia\predictions_...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.472945,0.340820,0.336541,0.197154,...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.414943,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.326770,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.252685,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.000346,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.053920
9

In [138]:
df_old

Index(['name', 'dataset', 'image_path', 'label_path', 'prediction_path',
       'similar_image_paths', 'similar_IoU_score', 'other_ratio', 'road_ratio',
       'sidewalk_ratio', 'vegetation_ratio', 'sky_ratio', 'car_ratio',
       'simple_tsne_1', 'simple_tsne_2', 'meaningful_tsne_1',
       'meaningful_tsne_2', 'pca_1', 'pca_2', 'umap_1', 'umap_2',
       'bottleneck_tsne_embedding', 'bottleneck_pca_embedding', 'overall_iou',
       'other_iou', 'road_iou', 'sidewalk_iou', 'vegetation_iou', 'sky_iou',
       'car_iou', 'similar_image_paths_others', 'similar_IoU_score_others',
       'similar_image_paths_road', 'similar_IoU_score_road',
       'similar_image_paths_sidewalk', 'similar_IoU_score_sidewalk',
       'similar_image_paths_vegetation', 'similar_IoU_score_vegetation',
       'similar_image_paths_sky', 'similar_IoU_score_sky',
       'similar_image_paths_car', 'similar_IoU_score_car'],
      dtype='object')

In [139]:
df_old["similar_image_paths_vegetation"][0]

'dataset\\domain_adaptation\\Synthia\\RGB_subset_500\\0003913.png'

In [180]:
df_old.drop(columns = ["prediction_path"])

,name,dataset,image_path,label_path,similar_image_paths,similar_IoU_score,other_ratio,road_ratio,sidewalk_ratio,vegetation_ratio,...,similar_image_paths_road,similar_IoU_score_road,similar_image_paths_sidewalk,similar_IoU_score_sidewalk,similar_image_paths_vegetation,similar_IoU_score_vegetation,similar_image_paths_sky,similar_IoU_score_sky,similar_image_paths_car,similar_IoU_score_car
id,,,,,,,,,,,,,,,,,,,,,
0,frankfurt_000000_000294_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.660623,0.528985,0.297838,0.080459,0.020084,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.669665,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.445261,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.526673,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.516886,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.220297
1,frankfurt_000000_000576_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.581997,0.384708,0.345542,0.019122,0.085221,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.625078,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.192217,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.338053,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.546513,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.328769
2,frankfurt_000000_001016_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.663334,0.505177,0.336215,0.023324,0.011662,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.603962,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.205325,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.126580,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.000000,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.309487
3,frankfurt_000000_001236_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.484486,0.179743,0.381479,0.009730,0.256977,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.685399,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.090810,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.425622,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.459195,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.343273
4,frankfurt_000000_001751_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.659385,0.405927,0.379869,0.024012,0.052703,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.682732,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.358655,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.569467,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.642079,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.213340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0009354.png,Synthia,dataset\domain_adaptation\Synthia\RGB_subset_5...,dataset\domain_adaptation\Synthia\new labels\0...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.472945,0.340820,0.336541,0.197154,0.094768,...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.414943,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.326770,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.252685,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.000346,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.053920
996,0009359.png,Synthia,dataset\domain_adaptation\Synthia\RGB_subset_5...,dataset\domain_adaptation\Synthia\new labels\0...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.380413,0.284762,0.331777,0.245871,0.107303,...,dataset\domain_adaptati

In [181]:
df_old["prediction_path"]=new_pred_paths_cityscapes

In [183]:
df_old["prediction_path"].iloc[0]

'dataset\\domain_adaptation\\Cityscapes\\outputs\\preds_new_color\\frankfurt_000000_000294_leftImg8bit.png'

# Still need to change
* prediction_path (need hours of running at first)

In [122]:
df_old["label_path"].iloc[579]

'dataset\\domain_adaptation\\Synthia\\new labels\\0001628.png'

In [184]:
df_old.to_csv("discreted_data_v16.csv",index = "id")